In [14]:
# these are the different representations of a graph
adjacency_list = {'HG':['CH', 'M', 'CB'], 'CH': ['HG','P', 'SRM'], 'CB': ['Q', 'HG'], 'M':['HG', 'SRM'], 'SRM': ['CH', 'M']}
adjacency_matrix = [
    [0, 1, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 0, 0, 0],
    [1, 0, 0, 0, 1],
    [0, 1, 0, 1, 0]

]
names = ['HG', 'CH', 'CB', 'M', 'SRM']
edge_lst = [('HG','CH'), ('HG','M'), ('HG','CB'), ('CH','HG'), ('CH','P'),('CH','SRM'), ('CB','Q'), ('CB','HG'), ('M','HG'), ('M','SRM'), ('SRM','CH'), ('SRM','M')]


#1 Write a code to find the degree of each vertex, and store it in a dictionary. Sort the keys of the dictionary by the degree stored in the values.

In [1]:
def func(data):
    degree_dict = {}

    # Case 1: Input is an adjacency list represented as a dictionary
    if type(data) == dict:
        for vertex in data:
            # Degree is the number of connections (neighbors)
            degree_dict[vertex] = len(data[vertex])

    # Case 2: Input is a list
    elif type(data) == list:
        # Case 2.1: List of edges (tuples)
        if type(data[0]) == tuple:
            names = ['HG', 'CH', 'CB', 'M', 'SRM']
            for name in names:
                count = 0
                for edge in data:
                    # Count if vertex appears in either end of the edge
                    if edge[0] == name or edge[1] == name:
                        count += 1
                degree_dict[name] = count

        # Case 2.2: Adjacency matrix
        elif type(data[0]) == list:
            names = ['HG', 'CH', 'CB', 'M', 'SRM']
            for i in range(len(names)):
                count = 0
                for j in range(len(names)):
                    # Check if there's a connection in the matrix
                    if data[i][j] == 1:
                        count += 1
                degree_dict[names[i]] = count

    # Sort the dictionary by degree value
    sorted_degree_dict = dict(sorted(degree_dict.items(), key=lambda x: x[1]))
    
    return sorted_degree_dict


#2 Write a code to inter-convert the 3 graph representations we have learnt

In [2]:
def adjacency_list_to_matrix(adjacency_list, names):
    size = len(names)
    matrix = [[0 for _ in range(size)] for _ in range(size)]

    for key in adjacency_list:
        if key in names:
            row = names.index(key)
            for neighbor in adjacency_list[key]:
                if neighbor in names:
                    col = names.index(neighbor)
                    matrix[row][col] = 1
    return matrix

In [3]:
def adjacency_matrix_to_edge_list(matrix, names):
    edge_list = []
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 1:
                edge_list.append((names[i], names[j]))
    return edge_list

In [4]:
def edge_list_to_adjacency_list(edge_list):
    adjacency_list = {}
    
    for u, v in edge_list:
        # Add edge u -> v
        if u not in adjacency_list:
            adjacency_list[u] = []
        adjacency_list[u].append(v)
        
        # Add edge v -> u (since the graph is undirected)
        if v not in adjacency_list:
            adjacency_list[v] = []
        adjacency_list[v].append(u)
    
    return adjacency_list

#3 Given a graph and two vertices, check if they are adjacent.

In [5]:
def check_adjacent(v1, v2, adj_list):
    for key_nodes in adj_list:  # go through each vertex in the graph
        if key_nodes == v1:  # when we find the first vertex
            for nb_nodes in adj_list[v1]:  # check all its neighbors
                if nb_nodes == v2:  # if the second vertex is a neighbor
                    return True  # they are adjacent
    return False  # not found, so they are not adjacent


#4 Check if a graph is complete.

In [6]:
def check_complete(adj_list):
    total_vertices = len(adj_list)  # count total number of vertices

    for key_nodes in adj_list:
        neighbors = set(adj_list[key_nodes])  # get the neighbors of the current vertex

        # In a complete graph, each vertex should be connected to all other vertices (except itself)
        if len(neighbors) != total_vertices - 1:
            return False  # not connected to all other vertices

    return True  # all vertices are connected to every other vertex


#5 Check if a graph is connected.

In [7]:
from collections import deque

# Function to check if the graph is connected
def is_connected(adj_list):
    visited = []  # to keep track of visited nodes
    queue = deque()  # initialize queue

    # Start BFS from the first node in the graph
    start_node = list(adj_list.keys())[0]
    queue.append(start_node)
    visited.append(start_node)

    while queue:
        current = queue.popleft()  # get one node from the queue
        for neighbor in adj_list[current]:  # go through all its neighbors
            if neighbor not in visited:
                visited.append(neighbor)
                queue.append(neighbor)  # add unvisited neighbors to the queue

    # After traversal, if all nodes were visited, graph is connected
    return len(visited) == len(adj_list)


#6 Given a graph and a list of vertices, check if it forms a walk, or a trail or a path or none of these.

In [8]:
def check_walk_trail_path_nota(seq, edge_list):
    # First, create a list of all edges including both directions (for undirected graph)
    all_edges = []
    for edge in edge_list:
        u, v = edge
        all_edges.append((u, v))
        all_edges.append((v, u))  # Add reverse direction

    # Function to check if the sequence is a walk
    def is_walk(seq):
        for i in range(len(seq) - 1):
            if (seq[i], seq[i+1]) not in all_edges:
                return False  # If even one edge is missing, it's not a walk
        return True

    # Function to check if the sequence is a trail (no repeated edges)
    def is_trail(seq):
        if is_walk(seq) == False:
            return False  # Must be a walk first
        seen_edges = []
        for i in range(len(seq) - 1):
            u = seq[i]
            v = seq[i+1]
            # To handle undirected edges, store as sorted pair
            edge = tuple(sorted([u, v]))
            if edge in seen_edges:
                return False  # Edge repeated → not a trail
            seen_edges.append(edge)
        return True

    # Function to check if the sequence is a path (no repeated vertices)
    def is_path(seq):
        if is_trail(seq) == False:
            return False  # Must be a trail first
        if len(seq) != len(set(seq)):
            return False  # Vertices repeated → not a path
        return True

    # Decide the type in order of strictness
    if is_path(seq):
        return "Path"
    elif is_trail(seq):
        return "Trail"
    elif is_walk(seq):
        return "Walk"
    else:
        return "None"


#7 Check if a given graph is a tree.

In [9]:
from collections import deque

def is_tree(adj_list):
    visited = []
    parent = {}
    queue = deque()

    # Start BFS from the first node
    start_node = list(adj_list.keys())[0]
    queue.append(start_node)
    visited.append(start_node)
    parent[start_node] = None  # root has no parent

    while queue:
        current = queue.popleft()
        for neighbor in adj_list[current]:
            if neighbor not in visited:
                visited.append(neighbor)
                queue.append(neighbor)
                parent[neighbor] = current
            elif parent[current] != neighbor:
                # If the neighbor is visited and not the parent, a cycle is found
                return False

    # If the number of visited nodes is not equal to total nodes, graph is not connected
    if len(visited) != len(adj_list):
        return False

    return True  # If no cycles and connected, it's a tree


#8 Given a connected cyclic graph, find its spanning tree

#9 Given a tree, find the number of leaf nodes

In [10]:
def count_leaf_nodes_directed(adj_list):
    leaf_count = 0

    # Go through each node and check if it has no outgoing edges
    for node in adj_list:
        if len(adj_list[node]) == 0:  # Node has no outgoing edges, it is a leaf
            leaf_count += 1

    return leaf_count


#10 Given a tree, check if it's a binary tree

In [11]:
def is_binary_tree(adj_list):
    for node in adj_list:
        if len(adj_list[node]) > 2:  # If any node has more than 2 children
            return False 
    return True 


#11 Given a tree and a node, find its height

In [12]:
def find_height(adj_list, node):
    # Helper function to recursively calculate the height
    def dfs(current_node):
        # If the node has no children, it's a leaf, so height is 0
        if current_node not in adj_list or len(adj_list[current_node]) == 0:
            return 0
        
        # Recursively calculate the height of all children and return the max height
        child_heights = [dfs(child) for child in adj_list[current_node]]
        return 1 + max(child_heights)  # Add 1 for the current node

    return dfs(node)


#12 Given a tree and a node, find its depth

In [13]:
def find_depth(adj_list, node):
    def dfs(current_node, depth):
        if current_node == node:  # Found the node
            return depth
        if current_node in adj_list:  # Recur for children
            for child in adj_list[current_node]:
                result = dfs(child, depth + 1)
                if result != -1:
                    return result
        return -1  # Node not found

    return dfs(list(adj_list.keys())[0], 0)  # Start from root
